In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import math
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

C:\Users\91996\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data=pd.read_csv("C:\B Drive\downloads\Bank_Personal_Loan_Modelling.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [3]:
data.drop("ZIP Code",axis=1,inplace=True)
data.drop("ID",axis=1,inplace=True)
data['target']=data['Personal Loan']
data.drop('Personal Loan',axis=1,inplace=True)


In [4]:
X=data.iloc[:,:-1]
y=data.iloc[:,-1]
print(X.shape, y.shape)
sc = StandardScaler()
X = sc.fit_transform(X)
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=0)

(5000, 11) (5000,)


In [5]:
max_node=5
num_hiddenlayers=2
nn_in=x_train.shape[1]
nn_hidden1=max_node
nn_hidden2=max_node
nn_out=1

In [6]:
def network(nn_in,nn_hidden1,nn_hidden2,nn_out):
    model=Sequential()
    model.add(Dense(nn_hidden1,input_dim=nn_in,activation='relu'))
    model.add(Dense(nn_hidden2,activation='relu'))
    model.add(Dense(nn_out,activation='linear'))
    model.compile(loss='mse',optimizer='adam',metrics=['mae'])
    return model


In [7]:
def run_episode(x_train,y_train,x_test,y_test,nn_in,nn_out,policy):
    nn_hidden1,nn_hidden2=policy
    model=network(nn_in,nn_hidden1,nn_hidden2,nn_out)
    model.fit(x_train,y_train,epochs=10,verbose=0,validation_split=0.05)
    _,accuracy=model.evaluate(x_test,y_test)
    return accuracy

In [8]:
policy=[5,2]
acci=run_episode(x_train,y_train,x_test,y_test,nn_in,nn_out,policy)
acci

47/47 [==============================] - 0s 1ms/step - loss: 0.0713 - mae: 0.1658


0.1658155769109726

In [9]:
def evaluate_policy(x_train,y_train,x_test,y_test,nn_in,nn_out,policy,n_episodes=1):
    total_rewards=0.0
    for _ in range(n_episodes):
        total_rewards+=1/run_episode(x_train,y_train,x_test,y_test,nn_in,nn_out,policy)
    return total_rewards/n_episodes

In [10]:
def gen_random_policy(max_node,num_hiddenlayers):
    return np.random.choice(max_node+1,size=(num_hiddenlayers))


In [11]:
def crossover(policy1,policy2,num_hiddenlayers):
    new_policy=policy1.copy()
    for i in range(num_hiddenlayers):
        rand=np.random.uniform()
        if rand>0.5:
            new_policy[i]=policy2[i]
    return new_policy

In [12]:
def mutation(policy,num_hiddenlayers,max_node,p=0.05):
    new_policy=policy.copy()
    for i in range(num_hiddenlayers):
        rand=np.random.uniform()
        if rand<p:
            new_policy[i]=np.random.choice(max_node+1)
    return new_policy

In [14]:
if __name__=='__main__':
    random.seed(1234)
    np.random.seed(1234)
    n_policy=10
    n_steps=5
    start=time.time()
    policy_pop=[gen_random_policy(max_node,num_hiddenlayers) for _ in range(n_policy)]
    for idx in range(n_steps):
        policy_scores=[evaluate_policy(x_train,y_train,x_test,y_test,nn_in,nn_out,p) for p in policy_pop]
        print("generation %d avgscore %0.3f" %(idx+1,sum(policy_scores)/len(policy_scores)))
        policy_ranks=list(reversed(np.argsort(policy_scores)))
        elite_set=[policy_pop[x] for x in policy_ranks[:5]]
        select_probs=np.array(policy_scores)/np.sum(policy_scores)
        if np.sum(policy_scores)==0:
            pp=1/np.array(policy_scores).size
            select_probs=pp*np.ones(np.array(policy_scores).size)
        child_set=[crossover(
            policy_pop[np.random.choice(range(n_policy),p=select_probs)],
            policy_pop[np.random.choice(range(n_policy),p=select_probs)],num_hiddenlayers) for _ in range(n_policy - 5)]
        mutated_list=[mutation(p,num_hiddenlayers,max_node) for p in child_set]
        policy_pop=elite_set
        policy_pop+=mutated_list
    policy_score=[evaluate_policy(x_train,y_train,x_test,y_test,nn_in,nn_out,p) for p in policy_pop]
    best_policy=policy_pop[np.argmax(policy_score)]
    
    end=time.time()
    print("average scores %0.2f"%(sum(policy_score)/len(policy_score)))
    print("best policy is ",best_policy)

47/47 [==============================] - 0s 2ms/step - loss: 0.0783 - mae: 0.1696
generation 1 avgscore 7.791
47/47 [==============================] - 0s 2ms/step - loss: 0.0782 - mae: 0.1677
generation 2 avgscore 8.626
47/47 [==============================] - 0s 2ms/step - loss: 0.0853 - mae: 0.0853
generation 3 avgscore 9.785
47/47 [==============================] - 0s 1ms/step - loss: 0.0783 - mae: 0.1680
generation 4 avgscore 10.240
47/47 [==============================] - 0s 1ms/step - loss: 0.0783 - mae: 0.1702
generation 5 avgscore 9.380
47/47 [==============================] - 0s 2ms/step - loss: 0.0853 - mae: 0.0853
average scores 11.72
best policy is  [0 0]
